In [1]:
!pip install geopandas fiona 

     |████████████████████████████████| 972kB 14.4MB/s 
     |████████████████████████████████| 14.8MB 214kB/s 
     |████████████████████████████████| 6.5MB 45.1MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#Goal here is to read in & access this ~1gb file
#File is in this area (of my drive, at least): /content/drive/My Drive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_grip4
# Trying: https://gis.stackexchange.com/questions/32762/how-to-access-feature-classes-in-file-geodatabases-with-python-and-gdal (look far down)

#Data taken from: https://www.globio.info/download-grip-dataset
#Some similar data here: https://enterprisecontentnew-usfs.hub.arcgis.com/datasets/motor-vehicle-use-map-roads-feature-layer?page=17611


import fiona 
import geopandas as gpd

#%%%%%%%%%%%%%%%%% We May Have Different Drive Paths To This Folder %%%%%%%%%%%%%%%%%%%%
gdb_file_path = '/content/drive/MyDrive/20F/ENGS108_Final_Project/Data_Sets/GRIP4_GlobalRoadsBackup.gdb/'


layers = fiona.listlayers(gdb_file_path)


for layername in fiona.listlayers(gdb_file_path):
    with fiona.open(gdb_file_path, layer=layername) as src:
        print(layername, len(src))
c = fiona.open(gdb_file_path, 'r')
print(len(src))
print(type(src))
print(len(c))
c[10060]



#gdf = gpd.read_file(gdb_file_path)
#gdf.head(2)

GRIP4_GlobalRoads 25758453
25758453
<class 'fiona.collection.Collection'>
25758453


{'geometry': {'coordinates': [[(-113.5190227, 53.607822000000056),
    (-113.51784859999998, 53.60790960000003),
    (-113.51777529999998, 53.60794720000007)]],
  'type': 'MultiLineString'},
 'id': '10060',
 'properties': OrderedDict([('GP_RTP', 5),
              ('GP_REX', 1),
              ('GP_RAV', 2),
              ('GP_RRG', 1),
              ('GP_RCY', 124),
              ('GP_RSE', 0),
              ('GP_RSI', 54),
              ('GP_RSY', 2014),
              ('Shape_Length', 0.001259744524104104),
              ('gp_gripreg', 1)]),
 'type': 'Feature'}

In [7]:
c[167]['geometry']['coordinates']

[[(-79.59239369999995, 43.197635700000035),
  (-79.59210729999995, 43.19831650000003)]]

In [10]:
import datetime
now = datetime.datetime.now()
print(now)

'''
Finds the indeces of c that are within our desired range
'''
def linear_grid_finder(clist,lat_min,lat_max,long_min,long_max, kind):
  idcs = []
  for i in range(1,len(clist)-1):
      if kind!=1:
        #print("latitude:", lat_min, clist[i]['geometry']['coordinates'][0][0][1], lat_max)
        #print("longitude:", long_min, clist[i]['geometry']['coordinates'][0][0][0], long_max)
        if lat_min < clist[i]['geometry']['coordinates'][0][0][1] < lat_max:
          if long_min < clist[i]['geometry']['coordinates'][0][0][0] < long_max:
            idcs.append(i)
      else:
         if lat_min < clist[i]['geometry']['coordinates'][0][0][0][1] < lat_max:
          if long_min < clist[i]['geometry']['coordinates'][0][0][0][0] < long_max:
            idcs.append(i)
  return idcs

now = datetime.datetime.now()
print(now)

2020-11-30 14:30:06.750428
2020-11-30 14:30:06.750689


In [11]:
'''
Splits a rectangular surface into smaller rectangles and returns them all
'''
def split_grid(num_rows,num_cols,x_min,x_max,y_min,y_max):
    quad_list = []
    increment_rows = (y_max - y_min)/num_rows
    increment_cols = (x_max - x_min)/num_cols

    for i in range(1,num_rows):
      for j in range(1,num_cols):
          quad_list.append((x_min+(i-1)*increment_cols, x_min+(i)*increment_cols, y_min+(j-1)*increment_rows, y_min+(j)*increment_rows))
    return quad_list

In [12]:
print(split_grid(10,10,0,100,0,100))

[(0.0, 10.0, 0.0, 10.0), (0.0, 10.0, 10.0, 20.0), (0.0, 10.0, 20.0, 30.0), (0.0, 10.0, 30.0, 40.0), (0.0, 10.0, 40.0, 50.0), (0.0, 10.0, 50.0, 60.0), (0.0, 10.0, 60.0, 70.0), (0.0, 10.0, 70.0, 80.0), (0.0, 10.0, 80.0, 90.0), (10.0, 20.0, 0.0, 10.0), (10.0, 20.0, 10.0, 20.0), (10.0, 20.0, 20.0, 30.0), (10.0, 20.0, 30.0, 40.0), (10.0, 20.0, 40.0, 50.0), (10.0, 20.0, 50.0, 60.0), (10.0, 20.0, 60.0, 70.0), (10.0, 20.0, 70.0, 80.0), (10.0, 20.0, 80.0, 90.0), (20.0, 30.0, 0.0, 10.0), (20.0, 30.0, 10.0, 20.0), (20.0, 30.0, 20.0, 30.0), (20.0, 30.0, 30.0, 40.0), (20.0, 30.0, 40.0, 50.0), (20.0, 30.0, 50.0, 60.0), (20.0, 30.0, 60.0, 70.0), (20.0, 30.0, 70.0, 80.0), (20.0, 30.0, 80.0, 90.0), (30.0, 40.0, 0.0, 10.0), (30.0, 40.0, 10.0, 20.0), (30.0, 40.0, 20.0, 30.0), (30.0, 40.0, 30.0, 40.0), (30.0, 40.0, 40.0, 50.0), (30.0, 40.0, 50.0, 60.0), (30.0, 40.0, 60.0, 70.0), (30.0, 40.0, 70.0, 80.0), (30.0, 40.0, 80.0, 90.0), (40.0, 50.0, 0.0, 10.0), (40.0, 50.0, 10.0, 20.0), (40.0, 50.0, 20.0, 30.0),

In [13]:
#idcs = linear_grid_finder(c,goal_lat_min,goal_lat_max, goal_long_min, goal_long_max,0)

def return_coords(clist, idx, kind):
  if kind == 1:
     return clist[idx]['geometry']['coordinates'][0][0]
  else:
     return clist[idx]['geometry']['coordinates'][0]

def return_all_coords(clist, idcs, kind=0):
  all_coords=[]
  for idx in idcs:
      if clist[idx]:
        if len(clist[idx]['geometry']['coordinates'])==1:
          all_coords.append(return_coords(clist,idx, kind))
        else:
          for i in range(len(clist[idx]['geometry']['coordinates'])):
              all_coords.append(clist[idx]['geometry']['coordinates'][i])
  return all_coords

In [15]:
##Need a roads filter to delete some of the extra roads to reduce linear_grid_finder search time:

def filter_roads(goal_lat_min, goal_lat_max, goal_long_min, goal_long_max, dataset):
  filtered_dataset = []
  for i in range(1,len(dataset),1):
      if goal_lat_min < dataset[i]['geometry']['coordinates'][0][0][1] < goal_lat_max:
        #print( dataset[i]['geometry']['coordinates'][0][0][1] )
        if goal_long_min < dataset[i]['geometry']['coordinates'][0][0][0] < goal_long_max:
          filtered_dataset.append(dataset[i])
  return filtered_dataset

In [16]:
!pip install fastkml pykml
from fastkml import kml
from pykml import parser
import re

     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
  Created wheel for fastkml: filename=fastkml-0.11-cp36-none-any.whl size=61869 sha256=17c3b966786941dcf8084c8ce5d1b2ce9fa4b089a66e2f7704b11c0bb82ba2aa
  Stored in directory: /root/.cache/pip/wheels/55/01/ea/6191eb73e0894743d02b33a2b1a570e85242844d810804fbf2
  Created wheel for pygeoif: filename=pygeoif-0.7-cp36-none-any.whl size=19249 sha256=66c91684d7e2ed887dae170c5d5d1d42b26fc1e084c2d70e901b7eb5d2839306
  Stored in directory: /root/.cache/pip/wheels/60/6e/a7/3d3eef59ac84a86663d0f5c5a92091f5056e9aeb6588c4de34
Successfully built fastkml pygeoif


In [17]:
import matplotlib.pyplot as plt

def prepare_plot(roads_data):
  prepared_dataX = []
  prepared_dataY = []
  for feature in roads_data:
    these_xs = []
    these_ys = []
    try:
      for x, y in feature:#apparently this is faster than *zip(): https://stackoverflow.com/questions/8081545/how-to-convert-list-of-tuples-to-multiple-lists
        these_xs.append(x)
        these_ys.append(y)
    except:
      for x, y in feature[0]:
        these_xs.append(x)
        these_ys.append(y)

    #print("prepared_dataX receives", these_xs)
    prepared_dataX.append(these_xs)

    #print("prepared_dataY receives", these_ys)
    prepared_dataY.append(these_ys)
  return prepared_dataX, prepared_dataY

#All done. Now we can graph this stuff

In [25]:
from matplotlib.pyplot import figure
from matplotlib import pyplot as plt
plt.ioff()
import numpy as np
def create_image(roadX, roadY, n):
  fig = figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k') #Resize to 1000x1000 images
  if roadX:
    for i in range(0, len(roadX)):
      tuple_setX = roadX[i]
      tuple_setY = roadY[i]
      for j in range(0, len(tuple_setX)-1, 1):
        plt.plot(tuple_setX[j:j+2], tuple_setY[j:j+2], '-', linewidth = 2, color = 'grey')
    plt.axis('off')
  #plt.savefig('/content/drive/My Drive/ENGS108_Final_Project/Data_Sets/our_images/roads_' +  str(n) + '.png', bbox_inches = 'tight')
  plt.savefig('/content/drive/MyDrive/20F/ENGS108_Final_Project/Data_Sets/onlyroads/'+ str(n) + '.png', bbox_inches = 'tight')

In [20]:
total=100

In [ ]:
goal_lat_min = 42 #bounds for germany/france/netherlands/belgium
goal_lat_max = 50 # next up: https://keras.io/examples/generative/pixelcnn/
x_cuts = int(abs(goal_lat_max-goal_lat_min)*50)
goal_long_min = 3
goal_long_max = 10
y_cuts = int(abs(goal_long_max-goal_long_min)*50)
list_of_coords = split_grid(x_cuts,y_cuts,goal_lat_min,goal_lat_max, goal_long_min, goal_long_max)
filtered_roads = filter_roads(goal_lat_min, goal_lat_max, goal_long_min, goal_long_max, c)
print("# of cubes we are splitting up:", len(list_of_coords))
for k in range(1,len(list_of_coords)):
    print('Im here')
    min_lat, max_lat, min_long, max_long = list_of_coords[k]
    print('here1')
    idcs = linear_grid_finder(filtered_roads, min_lat, max_lat, min_long, max_long, 0)
    print('here21')
    print("and we might've found some roads:", len(idcs))
    if len(idcs) > 0:
      total+=1
      print(k)
      all_coords = return_all_coords(c,idcs)
      roadX,roadY = prepare_plot(all_coords)
      create_image(roadX,roadY, k+total)

# of cubes we are splitting up: 139251
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 6
3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Streaming output truncated to the last 5000 lines.
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found some roads: 0
Im here
here1
here21
and we might've found so

In [ ]:
len(list_of_coords)

361